In [1]:
# | default_exp routes.jupyter

In [2]:
# | exporti
from enum import Enum

import os
import httpx
from functools import partial

import urllib

import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.get_data as gd

import domolibrary.client.DomoError as de
import domolibrary.utils.chunk_execution as ce

In [3]:
# | hide
import datetime as dt
import pandas as pd

[![Tutorial Video pt 2 of 4]](https://youtu.be/7aTHw7scsa8)

### GET Routes


In [4]:
# | export


class JupyterAPI_Error(de.DomoError):
    def __init__(
        self, status, response, domo_instance, parent_class=None, function_name=None
    ):
        super().__init__(
            status=status,
            message=response,
            domo_instance=domo_instance,
            parent_class=parent_class,
            function_name=function_name,
        )


class JupyterAPI_WorkspaceStarted(de.DomoError):
    def __init__(
        self, status, response, domo_instance, parent_class=None, function_name=None
    ):
        super().__init__(
            status=status,
            message=response,
            domo_instance=domo_instance,
            parent_class=parent_class,
            function_name=function_name,
        )

In [5]:
# | export


@gd.route_function
async def get_jupyter_workspace_by_id(
    auth,
    workspace_id,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
):
    url = f"https://{auth.domo_instance}.domo.com/api/datascience/v1/workspaces/{workspace_id}"

    res = await gd.get_data(
        url=url,
        method="GET",
        auth=auth,
        parent_class=parent_class,
        session=session,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        debug_api=debug_api,
    )

    if not res.is_success:
        raise JupyterAPI_Error(
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

In [6]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

workspace_id = "1cfe9db4-5937-4889-beb3-a311fc42f246"
res = await get_jupyter_workspace_by_id(workspace_id=workspace_id, auth=token_auth)
res.response

warning this token has not been validated by who_am_i, run get_auth_token first


{'id': '1cfe9db4-5937-4889-beb3-a311fc42f246',
 'name': 'learn_jupyter',
 'description': '',
 'created': '2024-02-22T17:13:54.653+00:00',
 'updated': '2024-03-19T17:23:49.017+00:00',
 'lastRun': '2024-03-19T17:23:49.017+00:00',
 'instances': [{'started': '2024-03-19T17:23:49.647+00:00',
   'lastActivity': '2024-03-19T17:24:26.258+00:00',
   'url': 'https://domo-community.jupyter-prod1.domodatascience.com/hub/auth_refresh?domo_hostname=https%3A%2F%2Fdomo-community.domo.com&next=https%3A%2F%2Fdomo-community.jupyter-prod1.domodatascience.com%2Fuser%2Fdomo-community-1893952720%2F1cfe9db4%2F',
   'status': 'RUNNING',
   'userId': 1893952720}],
 'inputConfiguration': [],
 'outputConfiguration': [{'dataSourceId': '84658e41-924e-499c-987b-f82dc10163cc',
   'alias': 'DomoStats - Accounts',
   'streamId': 1375}],
 'owner': 1893952720,
 'coOwners': [],
 'bootstrapKernel': False,
 'migrateFiles': False,
 'jupyterKernel': 'PYTHON_3_9',
 'cpu': 4.0,
 'memory': 32.0,
 'timeoutHours': 8,
 'instanceTyp

In [7]:
# | export


def parse_instance_service_location_and_prefix(instance: dict, domo_instance):
    url = instance["url"]

    query = urllib.parse.unquote(urllib.parse.urlparse(url).query)
    query = urllib.parse.urlparse(query.split("&")[1].replace("next=", ""))

    return {
        "service_location": query.netloc.replace(domo_instance, "")[1:],
        "service_prefix": query.path,
    }


async def get_workspace_auth_token_params(workspace_id, auth, return_raw: bool = False):
    """
    params are needed for authenticating requests inside the workspace environment
    Note: you'll also need a internally generated jupyter_token to authenticate requests
    returns { service_location , service_prefix}
    """
    res = await get_jupyter_workspace_by_id(workspace_id=workspace_id, auth=auth)

    open_instances = res.response.get("instances")

    if return_raw:
        return open_instances

    if not open_instances:
        raise JupyterAPI_WorkspaceStarted(
            status=res.status,
            response= "There are no open instances. Do you need to start the workspace?",
            domo_instance=auth.domo_instance,
            function_name=res.traceback_details.function_name,
        )

    return parse_instance_service_location_and_prefix(
        open_instances[0], auth.domo_instance
    )

In [8]:
#| eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

workspace_id = "1cfe9db4-5937-4889-beb3-a311fc42f246"
try:
    print(await get_workspace_auth_token_params(workspace_id=workspace_id, auth=token_auth))

except JupyterAPI_WorkspaceStarted as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
{'service_location': 'jupyter-prod1.domodatascience.com', 'service_prefix': '/user/domo-community-1893952720/1cfe9db4/'}


In [9]:
# | export


@gd.route_function
async def get_jupyter_workspaces(
    auth: dmda.DomoAuth,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop=1,
    debug_api: bool = False,
    debug_loop: bool = False,
):
    url = f"https://{auth.domo_instance}.domo.com/api/datascience/v1/search/workspaces"

    body = {
        "limit": 50,
        "offset": 0,
        "sortFieldMap": {"CREATED": "DESC"},
        "filters": [],
    }

    def arr_fn(res):
        return res.response["workspaces"]

    offset_params = {"limit": "limit", "offset": "offset"}

    res = await gd.looper(
        url=url,
        method="POST",
        limit=50,
        body=body,
        auth=auth,
        arr_fn=arr_fn,
        offset_params_in_body=True,
        offset_params=offset_params,
        parent_class=parent_class,
        session=session,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        debug_api=debug_api,
        debug_loop=debug_loop,
    )
    if not res.is_success:
        raise JupyterAPI_Error(
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

In [10]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res_domo_workspaces = await get_jupyter_workspaces(
    token_auth, debug_loop=False, debug_api=False
)
res_domo_workspaces.response[0]

warning this token has not been validated by who_am_i, run get_auth_token first


{'id': '54578d3f-b1a5-4383-b295-c1a72162942e',
 'name': 'Test',
 'description': '',
 'created': '2024-03-06T04:06:51.502+00:00',
 'updated': '2024-03-06T04:11:17.855+00:00',
 'lastRun': '2024-03-06T04:07:08.633+00:00',
 'instances': [],
 'inputConfiguration': [],
 'outputConfiguration': [],
 'owner': 617808774,
 'coOwners': [],
 'bootstrapKernel': False,
 'migrateFiles': False,
 'jupyterKernel': 'PYTHON_3_9',
 'cpu': 4.0,
 'memory': 32.0,
 'timeoutHours': 8,
 'sharingEnabled': False,
 'sharingEnabledBy': 0,
 'accountConfiguration': [],
 'collectionConfiguration': [],
 'fileshareConfiguration': []}

In [11]:
[res["name"] for res in res_domo_workspaces.response]

['Test',
 'Jupyter Test',
 'learn_jupyter',
 'abcdef',
 'The Office',
 'monit',
 'Oleksii - workspace',
 'test_asm',
 'jw',
 'Elliott Leonard Test',
 'Aditya Jain WS',
 "Jason's Jeans Python Space",
 't1',
 'Test',
 'Prueba',
 'KL | DP21 Workshop ',
 'VS | DP21 Workshop',
 'Tutorial']

In [12]:
# | export


@gd.route_function
async def start_jupyter_workspace(
    workspace_id,
    auth: dmda.DomoAuth,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop=1,
    debug_api: bool = False,
    return_raw: bool = False,
):
    url = f"https://{auth.domo_instance}.domo.com/api/datascience/v1/workspaces/{workspace_id}/instances"

    try:
        res = await gd.get_data(
            url=url,
            method="POST",
            auth=auth,
            parent_class=parent_class,
            session=session,
            num_stacks_to_drop=debug_num_stacks_to_drop,
            debug_api=debug_api,
        )

        if return_raw:
            return res

    except RuntimeError as e:
        return rgd.ResponseGetData(
            status=500,
            response=f"starting workspace, please wait - {e}",
            is_success=False,
        )

    if res.status == 500 or res.status == 403:
        raise JupyterAPI_Error(
            status=res.status,
            response=f"you may not have access to this workspace {workspace_id}, is it shared with you? || OR may already be started -- {res.response}",
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    if not res.is_success:
        raise JupyterAPI_Error(
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    res.response = "workspace started"
    return res

In [13]:
test_workspace = res_domo_workspaces.response[4]
print(test_workspace["name"])

try:
    print(
        await start_jupyter_workspace(
            workspace_id=test_workspace["id"],
            auth=token_auth,
            debug_api=False,
            return_raw=False,
        )
    )
except Exception as e:
    print(e)

The Office
🛑  JupyterAPI_Error 🛑 - function: start_jupyter_workspace || status 403 || you may not have access to this workspace 5edc482e-f181-4db2-b139-ca5d9dcf8b2f, is it shared with you? || OR may already be started -- Forbidden at domo-community


In [14]:
# | export


@gd.route_function
async def get_jupyter_content(
    auth: dmda.DomoAuth,
    content_path: str = "",
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
):
    url = f"https://{auth.domo_instance}.{auth.service_location}{auth.service_prefix}api/contents/{content_path}"

    res = await gd.get_data(
        url=f"{url}",
        method="GET",
        auth=auth,
        headers={"authorization": f"Token {auth.jupyter_token}"},
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )
    if not res.is_success:
        raise JupyterAPI_Error(
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

#### sample implementation of get_jupyter_content


In [15]:
# currently must scrape from the API
jupyter_token = "9meJVc62g4nVWvlD9qdJFgmO3Oxm0r"


token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

workspace_params = await get_workspace_auth_token_params(
    workspace_id=workspace_id, auth=token_auth
)

# converts a token_auth into DomoJupyterTokenAuth with a new generate_header function for authenticating jupyter requests
dj_auth = dmda.DomoJupyterTokenAuth.convert_auth(
    auth=token_auth, jupyter_token=jupyter_token, **workspace_params
)

dj_auth

warning this token has not been validated by who_am_i, run get_auth_token first


DomoJupyterTokenAuth(jupyter_token='9meJVc62g4nVWvlD9qdJFgmO3Oxm0r', service_location='jupyter-prod1.domodatascience.com', service_prefix='/user/domo-community-1893952720/1cfe9db4/', domo_instance='domo-community', token_name=None, is_valid_token=None, url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true')

In [16]:
#| eval : false
try:
    res = await get_jupyter_content(
        auth=dj_auth,
        # content_path = 'my_terrible_demo.txt',
        # content_path="datatypes.ipynb",
        debug_api=None,
    )

    print(res.response["content"][0:2])

except JupyterAPI_Error as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
🛑  JupyterAPI_Error 🛑 - function: get_jupyter_content || status 403 || Forbidden at domo-community


### update jupyter content

improve code maintenance by implementing a factory design patternx


In [17]:
# | exporti


def generate_update_jupyter_body__new_content_path(content_path):
    if not content_path:
        return None

    if "/" in content_path:
        return "/".join(content_path.split("/")[:-1])
    else:
        return ""


def generate_update_jupyter_body__text(body, content_path=None):
    body.update(
        {
            "format": "text",
            "type": "file",
            "path": generate_update_jupyter_body__new_content_path(content_path),
        }
    )
    return body


def generate_update_jupyter_body__ipynb(body, content_path=None):

    body.update(
        {
            "format": None,
            "type": "notebook",
            "path": generate_update_jupyter_body__new_content_path(content_path),
        }
    )
    return body


def generate_update_jupyter_body__directory(content_path, body):
    body.update(
        {
            "path": generate_update_jupyter_body__new_content_path(content_path),
            "format": None,
            "type": "directory",
        }
    )
    return body

In [18]:
# |export
class generate_update_jupyter_body_factory(Enum):
    IPYNB = partial(generate_update_jupyter_body__ipynb)
    DIRECTORY = partial(generate_update_jupyter_body__directory)
    TEXT = partial(generate_update_jupyter_body__text)

    @classmethod
    def from_text(cls, value):
        try:
            return cls[value.upper()].value
        except:
            return cls.TEXT.value
    
def generate_update_jupyter_body(
    new_content, content_path: str  # my_folder/datatypes.ipynb
):
    """factory to construct properly formed body"""

    content_name = os.path.normpath(content_path).split(os.sep)[-1]

    if "." in content_path:
        content_type = content_path.split(".")[-1]
    else:
        content_type = "directory"

    body = {
        "name": content_name,
        "content": new_content,
        "path": content_path,
    }
    return generate_update_jupyter_body_factory.from_text(content_type)(
        body=body, content_path=content_path
    )

In [19]:
generate_update_jupyter_body("hello world", "hi.md")

{'name': 'hi.md',
 'content': 'hello world',
 'path': '',
 'format': 'text',
 'type': 'file'}

In [20]:
generate_update_jupyter_body(None, "new_folder")

{'name': 'new_folder',
 'content': None,
 'path': '',
 'format': None,
 'type': 'directory'}

## CRUD Routes


In [21]:
# | export


@gd.route_function
async def update_jupyter_file(
    auth: dmda.DomoJupyterAuth,
    new_content,
    content_path: str = "",  # file name and location in jupyter
    debug_api: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
    session: httpx.AsyncClient = None,
):
    dmda.test_is_jupyter_auth(auth)

    body = generate_update_jupyter_body(new_content, content_path)

    content_path_split = os.path.normpath(content_path).split(os.sep)

    url = f"https://{auth.domo_instance}.{auth.service_location}{auth.service_prefix}api/contents/{'/'.join(content_path_split)}"

    res = await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        body=body,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        session=session,
    )

    if not res.is_success:
        raise JupyterAPI_Error(
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
        )

    return res

#### sample implementation of update_content


In [22]:
try:
    res = await update_jupyter_file(
        auth=dj_auth,
        content_path="new_folder",
        new_content=None,
        debug_api=False,
    )

    res.response

    res = await update_jupyter_file(
        auth=dj_auth,
        content_path=f"new_folder/updated_{dt.date.today()}.txt",
        new_content="jae is excellent at demoes",
        debug_api=False,
    )

    print(res.response)
    
except JupyterAPI_Error as e:
    print(e)

🛑  JupyterAPI_Error 🛑 - function: status 403 || Forbidden at domo-community


### recursive get_content

[![Tutorial Video pt 3 of 4]](https://youtu.be/l32qTj08bCY)


In [23]:
# | exporti


async def get_content_recursive(
    auth,
    all_rows,
    content_path,
    logs,
    res: rgd.ResponseGetData,
    obj: dict = None,
    is_recursive: bool = True,
    is_skip_recent_executions: bool = True,
    is_skip_default_files: bool = True,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_num_stacks_to_drop=0,
    parent_class=None,
    session: httpx.AsyncClient = None,
):
    # set path (on initial execution there is no object)

    res = await get_jupyter_content(
        auth=auth,
        content_path=content_path,
        debug_api=debug_api,
        parent_class=parent_class,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop + 1,
        session=session,
    )
    if return_raw:
        return res

    obj = res.response
    obj_name = obj["name"]
    obj_type = obj["type"]
    obj_path = obj["path"]
    obj_content = obj["content"] or []

    s = {"content_path": obj_path, "type": obj_type, "name": obj_name}

    if (is_skip_recent_executions and obj_path.startswith("recent_executions")) or (
        is_skip_default_files and obj_path.startswith("domo_jupyter_examples")
    ):
        res.response = all_rows
        res.logs = logs
        return res

    all_rows.append(obj)
    s.update({"is_append": True})
    logs.append(s)

    res.response = all_rows
    res.logs = logs

    if obj["type"] != "directory":
        return res

    s.update({"content": len(obj_content), "all_rows": len(all_rows)})
    logs.append(s)

    res.response = all_rows
    res.logs = logs

    if not is_recursive:
        return res

    if len(obj_content) > 0:
        await ce.gather_with_concurrency(
            *[
                get_content_recursive(
                    auth=auth,
                    content_path=content["path"],
                    all_rows=all_rows,
                    logs=logs,
                    res=res,
                    is_skip_recent_executions=is_skip_recent_executions,
                    is_skip_default_files=is_skip_default_files,
                    debug_api=debug_api,
                    debug_num_stacks_to_drop=debug_num_stacks_to_drop + 1,
                    parent_class=parent_class,
                    session=session,
                )
                for content in obj_content
            ],
            n=5
        )

    return res

In [24]:
# | export


@gd.route_function
async def get_content(
    auth: dmda.DomoJupyterAuth,
    content_path="",
    is_recursive: bool = True,
    is_skip_recent_executions: bool = True,
    is_skip_default_files: bool = True,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
):
    dmda.test_is_jupyter_auth(auth)

    all_rows = []
    logs = []
    res = None

    return await get_content_recursive(
        auth=auth,
        content_path=content_path,
        all_rows=all_rows,
        logs=logs,
        res=res,
        is_recursive=is_recursive,
        is_skip_recent_executions=is_skip_recent_executions,
        is_skip_default_files=is_skip_default_files,
        return_raw=return_raw,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )

#### sample implementation of get_content_recursive


In [25]:
# |eval: false

try:
    res = await get_content(
        auth=dj_auth,
        debug_api=False,
        return_raw=True,
        is_recursive=False,
        content_path="new_folder",
    )

    # print(
    #     "num_files",
    #     len([content for content in res.response if content["type"] != "directory"]),
    # )
    # pd.DataFrame(res.logs).fillna(0)
    pd.DataFrame(res.response)

except JupyterAPI_Error as e:
    print(e)

🛑  JupyterAPI_Error 🛑 - function: wrapper || status 403 || Forbidden at domo-community


In [26]:
# | hide
import nbdev

nbdev.nbdev_export("./jupyter.ipynb")